In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd

In [3]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')

#Define a list to store all the route data
Kerala_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            Kerala_Route.append({'routename':routename,'routelink':routelink})
            

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 3):
    scrape_page()
    if page_number < 2:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_KERALA_ROUTE = pd.DataFrame(Kerala_Route)
df_KERALA_ROUTE

In [13]:
#retrive the bus details
driver_KE = webdriver.Chrome()
Bus_names_KE = []
Bus_types_KE = []
Start_Time_KE = []
End_Time_KE = []
Ratings_KE = []
Total_Duration_KE = []
Prices_KE = []
Seats_Available_KE = []
Route_names = []
Route_links = []

for i,r in df_KERALA_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_KE.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_KE.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_KE.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_KE.execute_script("return document.body.scrollHeight")
    while True:
        driver_KE.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_KE.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_KE.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_KE.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_KE.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_KE.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_KE.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_KE.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_KE.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_KE.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_KE.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_KE.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_KE.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_KE.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_KE.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_KE.append(ratings.text)
    for price_elem in price:
        Prices_KE.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_KE.append(seats_elem.text)

In [14]:
KERALA_DATA = {
    'Bus_name': Bus_names_KE,
    'Bus_type': Bus_types_KE,
    'Start_time': Start_Time_KE,
    'End_time': End_Time_KE,
    'Total_duration': Total_Duration_KE,
    'Price': Prices_KE,
    "Seats_Available":Seats_Available_KE,
    "Ratings":Ratings_KE,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
KERALA_DATA_1 = pd.DataFrame(KERALA_DATA)
KERALA_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/KE_BUS1.CSV",index=False)
KERALA_DATA_1


In [2]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')

#Define a list to store all the route data
AP_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            AP_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_AP_ROUTE = pd.DataFrame(AP_Route)
df_AP_ROUTE

In [8]:
#retrive the bus details
driver_AP = webdriver.Chrome()
Bus_names_AP = []
Bus_types_AP = []
Start_Time_AP = []
End_Time_AP = []
Ratings_AP = []
Total_Duration_AP = []
Prices_AP = []
Seats_Available_AP = []
Route_names = []
Route_links = []

for i,r in df_AP_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_AP.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_AP.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_AP.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_AP.execute_script("return document.body.scrollHeight")
    while True:
        driver_AP.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_AP.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_AP.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_AP.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_AP.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_AP.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_AP.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_AP.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_AP.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_AP.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_AP.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_AP.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_AP.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_AP.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_AP.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_AP.append(ratings.text)
    for price_elem in price:
        Prices_AP.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_AP.append(seats_elem.text)

In [9]:
AP_DATA = {
    'Bus_name': Bus_names_AP,
    'Bus_type': Bus_types_AP,
    'Start_time': Start_Time_AP,
    'End_time': End_Time_AP,
    'Total_duration': Total_Duration_AP,
    'Price': Prices_AP,
    "Seats_Available":Seats_Available_AP,
    "Ratings":Ratings_AP,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
AP_DATA_1 = pd.DataFrame(AP_DATA)
AP_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/AP_BUS1.CSV",index=False)
AP_DATA_1

In [17]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile')

#Define a list to store all the route data
TS_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            TS_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1,4):
    scrape_page()
    if page_number < 3:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_TS_ROUTE = pd.DataFrame(TS_Route)
df_TS_ROUTE

In [19]:
#retrive the bus details
driver_TS = webdriver.Chrome()
Bus_names_TS = []
Bus_types_TS = []
Start_Time_TS = []
End_Time_TS = []
Ratings_TS = []
Total_Duration_TS = []
Prices_TS = []
Seats_Available_TS = []
Route_names = []
Route_links = []

for i,r in df_TS_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_TS.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_TS.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_TS.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_TS.execute_script("return document.body.scrollHeight")
    while True:
        driver_TS.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_TS.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_TS.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_TS.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_TS.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_TS.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_TS.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_TS.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_TS.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_TS.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_TS.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_TS.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_TS.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_TS.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_TS.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_TS.append(ratings.text)
    for price_elem in price:
        Prices_TS.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_TS.append(seats_elem.text)

In [20]:
TS_DATA = {
    'Bus_name': Bus_names_TS,
    'Bus_type': Bus_types_TS,
    'Start_time': Start_Time_TS,
    'End_time': End_Time_TS,
    'Total_duration': Total_Duration_TS,
    'Price': Prices_TS,
    "Seats_Available":Seats_Available_TS,
    "Ratings":Ratings_TS,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
TS_DATA_1 = pd.DataFrame(TS_DATA)
TS_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/TS_BUS1.CSV",index=False)
TS_DATA_1

In [3]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')

#Define a list to store all the route data
RS_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            RS_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1,4):
    scrape_page()
    if page_number < 3:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_RS_ROUTE = pd.DataFrame(RS_Route)
df_RS_ROUTE

In [6]:
#retrive the bus details
driver_RS = webdriver.Chrome()
Bus_names_RS = []
Bus_types_RS = []
Start_Time_RS = []
End_Time_RS = []
Ratings_RS = []
Total_Duration_RS = []
Prices_RS = []
Seats_Available_RS = []
Route_names = []
Route_links = []

for i,r in df_RS_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_RS.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_RS.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_RS.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_RS.execute_script("return document.body.scrollHeight")
    while True:
        driver_RS.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_RS.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_RS.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_RS.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_RS.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_RS.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_RS.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_RS.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_RS.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_RS.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_RS.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_RS.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_RS.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_RS.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_RS.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_RS.append(ratings.text)
    for price_elem in price:
        Prices_RS.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_RS.append(seats_elem.text)

In [7]:
RS_DATA = {
    'Bus_name': Bus_names_RS,
    'Bus_type': Bus_types_RS,
    'Start_time': Start_Time_RS,
    'End_time': End_Time_RS,
    'Total_duration': Total_Duration_RS,
    'Price': Prices_RS,
    "Seats_Available":Seats_Available_RS,
    "Ratings":Ratings_RS,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
RS_DATA_1 = pd.DataFrame(RS_DATA)
RS_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/RS_BUS1.CSV",index=False)
RS_DATA_1

In [9]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile')

#Define a list to store all the route data
SB_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            SB_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1,6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_SB_ROUTE = pd.DataFrame(SB_Route)
df_SB_ROUTE

In [11]:
#retrive the bus details
driver_SB = webdriver.Chrome()
Bus_names_SB = []
Bus_types_SB = []
Start_Time_SB = []
End_Time_SB = []
Ratings_SB = []
Total_Duration_SB = []
Prices_SB = []
Seats_Available_SB = []
Route_names = []
Route_links = []

for i,r in df_SB_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_SB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_SB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_SB.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_SB.execute_script("return document.body.scrollHeight")
    while True:
        driver_SB.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_SB.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_SB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_SB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_SB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_SB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_SB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_SB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_SB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_SB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_SB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_SB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_SB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_SB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_SB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_SB.append(ratings.text)
    for price_elem in price:
        Prices_SB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_SB.append(seats_elem.text)

In [12]:
SB_DATA = {
    'Bus_name': Bus_names_SB,
    'Bus_type': Bus_types_SB,
    'Start_time': Start_Time_SB,
    'End_time': End_Time_SB,
    'Total_duration': Total_Duration_SB,
    'Price': Prices_SB,
    "Seats_Available":Seats_Available_SB,
    "Ratings":Ratings_SB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
SB_DATA_1 = pd.DataFrame(SB_DATA)
SB_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/SB_BUS1.CSV",index=False)
SB_DATA_1

In [14]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/astc/?utm_source=rtchometile')

#Define a list to store all the route data
AS_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            AS_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_AS_ROUTE = pd.DataFrame(AS_Route)
df_AS_ROUTE

In [20]:
#retrive the bus details
driver_AS = webdriver.Chrome()
Bus_names_AS = []
Bus_types_AS = []
Start_Time_AS = []
End_Time_AS = []
Ratings_AS = []
Total_Duration_AS = []
Prices_AS = []
Seats_Available_AS = []
Route_names = []
Route_links = []

for i,r in df_AS_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_AS.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_AS.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_AS.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_AS.execute_script("return document.body.scrollHeight")
    while True:
        driver_AS.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_AS.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_AS.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_AS.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_AS.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_AS.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_AS.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_AS.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_AS.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_AS.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_AS.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_AS.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_AS.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_AS.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_AS.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_AS.append(ratings.text)
    for price_elem in price:
        Prices_AS.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_AS.append(seats_elem.text)

In [21]:
AS_DATA = {
    'Bus_name': Bus_names_AS,
    'Bus_type': Bus_types_AS,
    'Start_time': Start_Time_AS,
    'End_time': End_Time_AS,
    'Total_duration': Total_Duration_AS,
    'Price': Prices_AS,
    "Seats_Available":Seats_Available_AS,
    "Ratings":Ratings_AS,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
AS_DATA_1 = pd.DataFrame(AS_DATA)
AS_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/AS_BUS1.CSV",index=False)
AS_DATA_1

In [23]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile')

#Define a list to store all the route data
UP_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            UP_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_UP_ROUTE = pd.DataFrame(UP_Route)
df_UP_ROUTE

In [25]:
#retrive the bus details
driver_UP = webdriver.Chrome()
Bus_names_UP = []
Bus_types_UP = []
Start_Time_UP = []
End_Time_UP = []
Ratings_UP = []
Total_Duration_UP = []
Prices_UP = []
Seats_Available_UP = []
Route_names = []
Route_links = []

for i,r in df_UP_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_UP.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_UP.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_UP.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_UP.execute_script("return document.body.scrollHeight")
    while True:
        driver_UP.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_UP.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_UP.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_UP.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_UP.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_UP.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_UP.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_UP.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_UP.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_UP.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_UP.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_UP.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_UP.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_UP.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_UP.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_UP.append(ratings.text)
    for price_elem in price:
        Prices_UP.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_UP.append(seats_elem.text)

In [26]:
UP_DATA = {
    'Bus_name': Bus_names_UP,
    'Bus_type': Bus_types_UP,
    'Start_time': Start_Time_UP,
    'End_time': End_Time_UP,
    'Total_duration': Total_Duration_UP,
    'Price': Prices_UP,
    "Seats_Available":Seats_Available_UP,
    "Ratings":Ratings_UP,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
UP_DATA_1 = pd.DataFrame(UP_DATA)
UP_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/UP_BUS1.CSV",index=False)
UP_DATA_1

In [29]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/north-bengal-state-transport-corporation')

#Define a list to store all the route data
NB_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            NB_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_NB_ROUTE = pd.DataFrame(NB_Route)
df_NB_ROUTE

In [31]:
#retrive the bus details
driver_NB = webdriver.Chrome()
Bus_names_NB = []
Bus_types_NB = []
Start_Time_NB = []
End_Time_NB = []
Ratings_NB = []
Total_Duration_NB = []
Prices_NB = []
Seats_Available_NB = []
Route_names = []
Route_links = []

for i,r in df_NB_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_NB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_NB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_NB.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_NB.execute_script("return document.body.scrollHeight")
    while True:
        driver_NB.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_NB.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_NB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_NB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_NB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_NB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_NB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_NB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_NB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_NB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_NB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_NB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_NB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_NB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_NB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_NB.append(ratings.text)
    for price_elem in price:
        Prices_NB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_NB.append(seats_elem.text)

In [32]:
NB_DATA = {
    'Bus_name': Bus_names_NB,
    'Bus_type': Bus_types_NB,
    'Start_time': Start_Time_NB,
    'End_time': End_Time_NB,
    'Total_duration': Total_Duration_NB,
    'Price': Prices_NB,
    "Seats_Available":Seats_Available_NB,
    "Ratings":Ratings_NB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
NB_DATA_1 = pd.DataFrame(NB_DATA)
NB_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/NB_BUS1.CSV",index=False)
NB_DATA_1

In [34]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu')

#Define a list to store all the route data
CD_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            CD_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_CD_ROUTE = pd.DataFrame(CD_Route)
df_CD_ROUTE

In [36]:
#retrive the bus details
driver_CD = webdriver.Chrome()
Bus_names_CD = []
Bus_types_CD = []
Start_Time_CD = []
End_Time_CD = []
Ratings_CD = []
Total_Duration_CD = []
Prices_CD = []
Seats_Available_CD = []
Route_names = []
Route_links = []

for i,r in df_CD_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_CD.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_CD.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_CD.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_CD.execute_script("return document.body.scrollHeight")
    while True:
        driver_CD.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_CD.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_CD.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_CD.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_CD.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_CD.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_CD.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_CD.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_CD.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_CD.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_CD.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_CD.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_CD.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_CD.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_CD.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_CD.append(ratings.text)
    for price_elem in price:
        Prices_CD.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_CD.append(seats_elem.text)

In [37]:
CD_DATA = {
    'Bus_name': Bus_names_CD,
    'Bus_type': Bus_types_CD,
    'Start_time': Start_Time_CD,
    'End_time': End_Time_CD,
    'Total_duration': Total_Duration_CD,
    'Price': Prices_CD,
    "Seats_Available":Seats_Available_CD,
    "Ratings":Ratings_CD,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
CD_DATA_1 = pd.DataFrame(CD_DATA)
CD_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/CD_BUS1.CSV",index=False)
CD_DATA_1

In [42]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get('https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile')

#Define a list to store all the route data
PUNJAB_Route = []

def scrape_page():
    # Locate elements  (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename= route.find_element(By.CLASS_NAME,'route').text
            routelink= route.find_element(By.CLASS_NAME,'route').get_attribute('href')
            #Then  Append extracted data to list
            PUNJAB_Route.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            continue

# Scrape data from the first 5 pages
for page_number in range(1,4):
    scrape_page()
    if page_number < 3:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling
 

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            break

In [ ]:
df_PUNJAB_ROUTE = pd.DataFrame(PUNJAB_Route)
df_PUNJAB_ROUTE


In [44]:
#retrive the bus details
driver_PUNJAB = webdriver.Chrome()
Bus_names_PUNJAB = []
Bus_types_PUNJAB = []
Start_Time_PUNJAB = []
End_Time_PUNJAB = []
Ratings_PUNJAB = []
Total_Duration_PUNJAB = []
Prices_PUNJAB = []
Seats_Available_PUNJAB = []
Route_names = []
Route_links = []

for i,r in df_PUNJAB_ROUTE.iterrows():
    link=r["routelink"]
    routes=r["routename"]

# Loop through each link
    driver_PUNJAB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_PUNJAB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_PUNJAB.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    last_height = driver_PUNJAB.execute_script("return document.body.scrollHeight")
    while True:
        driver_PUNJAB.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_PUNJAB.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_PUNJAB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_PUNJAB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_PUNJAB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_PUNJAB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_PUNJAB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_PUNJAB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_PUNJAB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_PUNJAB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_PUNJAB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_PUNJAB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_PUNJAB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_PUNJAB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_PUNJAB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_PUNJAB.append(ratings.text)
    for price_elem in price:
        Prices_PUNJAB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_PUNJAB.append(seats_elem.text)

In [45]:
PUNJAB_DATA = {
    'Bus_name': Bus_names_PUNJAB,
    'Bus_type': Bus_types_PUNJAB,
    'Start_time': Start_Time_PUNJAB,
    'End_time': End_Time_PUNJAB,
    'Total_duration': Total_Duration_PUNJAB,
    'Price': Prices_PUNJAB,
    "Seats_Available":Seats_Available_PUNJAB,
    "Ratings":Ratings_PUNJAB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

In [ ]:
PUNJAB_DATA_1 = pd.DataFrame(PUNJAB_DATA)
PUNJAB_DATA_1.to_csv("C:/Users/Admin/Desktop/REDBUS/ALLBUS/PUNJAB_BUS1.CSV",index=False)
PUNJAB_DATA_1